In [47]:
import gurobipy as gp
from gurobipy import GRB

- 决策变量：$x_{ij}$: 生产第i种糖用Ingredientj的量
- 目标函数：价格-成本
- 约束：可供Ingredient、成分、订单要求

In [48]:
combos, profit = gp.multidict(
    {
        ("MilkCandy", "IngredientA"): 24 - 20,
        ("MilkCandy", "IngredientB"): 24 - 12,
        ("MilkCandy", "IngredientC"): 24 - 8,
        ("FruitCandy", "IngredientA"): 15 - 20,
        ("FruitCandy", "IngredientB"): 15 - 12,
        ("FruitCandy", "IngredientC"): 15 - 8,
    }
)

In [49]:
m = gp.Model("糖果生产分配问题")

In [50]:
x = m.addVars(combos, name="x")

In [51]:
m.setObjective(x.prod(profit), sense=GRB.MAXIMIZE)

In [52]:
m.addConstr(
    x["MilkCandy", "IngredientA"] >= 0.5 * x.sum("MilkCandy", "*"), name="成分要求1"
)
m.addConstr(
    x["MilkCandy", "IngredientB"] >= 0.25 * x.sum("MilkCandy", "*"), name="成分要求2"
)
m.addConstr(
    x["MilkCandy", "IngredientC"] <= 0.1 * x.sum("MilkCandy", "*"), name="成分要求3"
)
m.addConstr(
    x["FruitCandy", "IngredientA"] <= 0.4 * x.sum("FruitCandy", "*"), name="成分要求4"
)
m.addConstr(
    x["FruitCandy", "IngredientB"] <= 0.4 * x.sum("FruitCandy", "*"), name="成分要求5"
)
m.addConstr(
    x["FruitCandy", "IngredientC"] >= 0.15 * x.sum("FruitCandy", "*"),
    name="成分要求6",
)

<gurobi.Constr *Awaiting Model Update*>

In [53]:
m.addConstr(x.sum("MilkCandy", "*") >= 600, name="订单要求1")
m.addConstr(x.sum("FruitCandy", "*") >= 800, name="订单要求2")

<gurobi.Constr *Awaiting Model Update*>

In [54]:
m.addConstr(x.sum("*", "IngredientA") <= 500, name="原料限制1")
m.addConstr(x.sum("*", "IngredientB") <= 750, name="原料限制2")
m.addConstr(x.sum("*", "IngredientC") <= 620, name="原料限制3")

<gurobi.Constr *Awaiting Model Update*>

In [55]:
m.write("assign.lp")